# import

In [1]:
# IMPORTS
from distutils.command.config import config
import pickle5 as pickle

# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import wandb
import os
from os.path import exists

# Evaluation
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_auc_score, \
    accuracy_score, f1_score, recall_score, precision_score, confusion_matrix

import wandb
import glob

# data
import numpy as np
import matplotlib.pyplot as plt

#torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from torch.utils.data import WeightedRandomSampler

# Load data

In [8]:
# CURR_PATH = os.getcwd()
CURR_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/LSTM'
DATA_PATH = '/home/svetlanamaslenkova/Documents/data/physionet.org/files/mimiciv/2.0/'
DF_PATH = CURR_PATH +'/dataframes_2/'
# ED_PATH = DATA_PATH+'physionet.org/files/mimic-iv-ed/1.0/ed/'
PKL_PATH = CURR_PATH+'/pickles_2/'
DF_PATH = CURR_PATH+'/dataframes_2/'
ICU_PATH = DATA_PATH + 'icu/'
HOSP_PATH = DATA_PATH + 'hosp/'
XGB_DATA_PATH = '/home/svetlanamaslenkova/Documents/AKI_deep/xgb/dataframes/'

In [15]:
with open(PKL_PATH + 'multihot_meds_icu_day_0.pkl', 'rb') as f:
    data_medications = pickle.load(f)
print(data_medications.stay_id.unique().shape[0])
data_medications.head()

70205


,subject_id,hadm_id,stay_id,med_ACD-A Citrate (1000ml),med_ACD-A Citrate (500ml),med_Abciximab (Reopro),med_Acetaminophen-IV,med_Acetylcysteine,med_Adenosine,med_Alteplase (TPA),...,med_Ranitidine (Prophylaxis),med_Rocuronium,med_Sodium Bicarbonate 8.4%,med_Thiamine,med_Tirofiban (Aggrastat),med_Treprostinil (Remodulin),med_Vasopressin,med_Vecuronium,med_Verapamil,med_Vitamin K (Phytonadione)
0,10000980,26913865,39765666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,225911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001217,24597018,37067082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001217,27703517,34592300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001725,25563031,31205490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001884,26184834,37510196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
with open(PKL_PATH + 'multihot_labs_icu_day_0.pkl', 'rb') as f:
    data_labs = pickle.load(f)
data_labs['stay_id'] = data_labs['stay_id'].astype('int64')
print(data_labs.stay_id.unique().shape[0])
data_labs.head()

75393


,subject_id,hadm_id,stay_id,icu_day_id,first_ALT,first_AST,first_AT (Antithrombin funct),first_Absolute Count - Basos,first_Absolute Count - Eos,first_Absolute Count - Lymphs,...,last_Triglyceride,last_Troponin-T,last_UF Heparin (Anti-Xa),last_Uric Acid,last_Vancomycin (Peak),last_Vancomycin (Random),last_Vancomycin (Trough),last_Venous CO2 Pressure,last_Venous O2 Pressure,last_WBC
0,10000032,29079034,39553978,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000980,26913865,39765666,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001217,24597018,37067082,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0
3,10001217,27703517,34592300,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4
4,10001725,25563031,31205490,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.1


In [97]:
with open(PKL_PATH + 'multihot_vitals_icu_day_0.pkl', 'rb') as f:
    data_vitals = pickle.load(f)
data_vitals['stay_id'] = data_vitals['stay_id'].astype('int64')
print(data_vitals.stay_id.unique().shape[0])
data_vitals.head()

36841


,subject_id,hadm_id,stay_id,icu_day_id,first_ABPd,first_ABPm,first_ABPs,first_ART BP Diastolic,first_ART BP Mean,first_ART BP Systolic,...,max_Temperature C,max_Temperature F,max_obpd lying,max_obps lying,max_obps sitting,max_obps standing,max_ohr lying,max_ohr sitting,max_ohr standing,max_rhr aar
0,10001884,26184834,37510196,0,NaN,NaN,NaN,NaN,NaN,NaN,...,35.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10002155,23822395,33685454,0,59.0,82.0,115.0,NaN,NaN,NaN,...,NaN,95.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002155,28994087,31090461,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10002348,22725460,32610785,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10002428,20321825,34807493,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
with open(XGB_DATA_PATH + 'data_output_xgb.pkl', 'rb') as f:
    data_outputs = pickle.load(f)
data_outputs['stay_id'] = data_outputs['stay_id'].astype('int64')
data_outputs = data_outputs[(data_outputs.icu_day_id==0)&(data_outputs.stay_id.isin(stays))]
print(data_outputs.stay_id.unique().shape[0])
data_outputs.head()

14077


abbreviation,subject_id,hadm_id,stay_id,icu_day_id,output
26,10002155,28994087,31090461,0,3260.0
33,10002348,22725460,32610785,0,325.0
42,10002428,23473524,35479615,0,2255.0
53,10002428,28662225,33987268,0,805.0
58,10002428,28662225,38875437,0,1630.0


In [111]:
with open(XGB_DATA_PATH + 'data_medications_xgb.pkl', 'rb') as f:
    data_medications = pickle.load(f)
data_medications['stay_id'] = data_medications['stay_id'].astype('int64')
data_medications = data_medications[(data_medications.stay_id.isin(stays))]
print(data_medications.stay_id.unique().shape[0])
data_medications.head()

14282


,stay_id,M01,B01,C03,C02
2,31090461,0,1,0,0
3,32610785,0,0,0,0
5,35479615,0,1,0,0
6,33987268,0,1,0,0
7,38875437,0,0,0,0


In [90]:
with open(XGB_DATA_PATH + 'data_demographics_xgb.pkl', 'rb') as f:
    data_demographics = pickle.load(f)
data_demographics = data_demographics[data_demographics.stay_id.isin(stays)]
print(data_demographics.stay_id.unique().shape[0])
data_demographics.head()

14281


,subject_id,hadm_id,stay_id,race,gender,age,BMI,Height,Weight,diastolic_bp,systolic_bp
110,10002155,28994087,31090461,1,1,82,21.4,60.0,109.4,51.0,99.0
121,10002348,22725460,32610785,1,1,77,NaN,NaN,NaN,NaN,NaN
123,10002428,23473524,35479615,1,1,81,20.0,59.0,99.0,78.0,131.0
127,10002428,28662225,33987268,1,1,81,20.0,59.0,99.0,78.0,131.0
128,10002428,28662225,33987268,1,1,81,20.0,59.0,99.0,78.0,131.0


In [130]:
with open(XGB_DATA_PATH + 'data_xgb.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset.columns = [col.replace(' ', '_') for col in dataset.columns]
dataset.columns = dataset.columns.str.lower()
print(dataset.shape)
dataset.head()

(33805, 65)


,subject_id,hadm_id,stay_id,bmi,height,weight,diastolic_bp,systolic_bp,min_sodium_(serum),max_chloride_(serum),...,min_nbpm,max_nbpd,min_nbpd,output,race,gender,age,aki_1,aki_2,aki_3
0,10002155,28994087,31090461,21.4,60.0,109.4,51.0,99.0,136.0,106.0,...,76.0,25.0,64.0,3260.0,1,1,82,0,0,0
1,10002428,20321825,34807493,20.0,59.0,99.0,78.0,131.0,133.0,93.0,...,112.0,32.0,101.0,888.0,1,1,81,0,0,0
2,10002428,23473524,35479615,20.0,59.0,99.0,78.0,131.0,135.0,107.0,...,NaN,NaN,NaN,2255.0,1,1,81,0,0,0
3,10002428,28662225,33987268,20.0,59.0,99.0,78.0,131.0,133.0,98.0,...,92.0,19.0,90.0,805.0,1,1,81,1,1,0
5,10002428,28662225,38875437,20.0,59.0,99.0,78.0,131.0,144.0,110.0,...,127.0,30.0,116.0,1630.0,1,1,81,1,1,0


In [155]:
dataset_demo = dataset[['subject_id',	'hadm_id',	'stay_id', 'bmi',	'height',	'weight',	'diastolic_bp',	'systolic_bp', 'race',	'gender',	'age']]
dataset_demo['gender'] = dataset_demo['gender'].astype(int)
dataset_demo['age'] = dataset_demo['age'].astype(int)

/home/svetlanamaslenkova/anaconda_envs/aki_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/svetlanamaslenkova/anaconda_envs/aki_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
dataset_demo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33805 entries, 0 to 41469
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   subject_id    33805 non-null  int64  
 1   hadm_id       33805 non-null  int64  
 2   stay_id       33805 non-null  int64  
 3   bmi           14263 non-null  float64
 4   height        12108 non-null  float64
 5   weight        15510 non-null  float64
 6   diastolic_bp  13847 non-null  float64
 7   systolic_bp   13847 non-null  float64
 8   race          33805 non-null  int64  
 9   gender        33805 non-null  object 
 10  age           33805 non-null  object 
dtypes: float64(5), int64(4), object(2)
memory usage: 3.1+ MB


In [24]:
with open(DF_PATH + 'pid_train_dataset_icu.pkl', 'rb') as f:
    pid_train_dataset_icu = pickle.load(f)

with open(DF_PATH + 'pid_test_dataset_icu.pkl', 'rb') as f:
    pid_test_dataset_icu = pickle.load(f)

with open(DF_PATH + 'pid_val_dataset_icu.pkl', 'rb') as f:
    pid_val_dataset_icu = pickle.load(f)

In [25]:
stays = [*list(pid_train_dataset_icu.stay_id.unique()), *list(pid_test_dataset_icu.stay_id.unique()),*list(pid_val_dataset_icu.stay_id.unique())]

In [112]:
train_stays = pid_train_dataset_icu.stay_id.unique()
test_stays = pid_test_dataset_icu.stay_id.unique()
val_stays = pid_val_dataset_icu.stay_id.unique()

In [26]:
len(stays)

14282

# Preproccesing

## Filter Missing

### Labs

In [98]:
data_labs = data_labs[data_labs.stay_id.isin(stays)]
print(data_labs.stay_id.unique().shape[0])

14246


In [56]:
for k,v in data_labs.isna().sum().sort_values(ascending=False).iteritems():
    print(k,v)

last_ScvO2 Central Venous O2% Sat 14246
first_Vancomycin (Peak) 14246
last_Vancomycin (Peak) 14246
first_ScvO2 Central Venous O2% Sat 14246
first_Tobramycin (Peak) 14244
last_Tobramycin (Peak) 14244
last_Gentamicin (Peak) 14243
first_Gentamicin (Peak) 14243
last_Phenytoin (Free) 14237
first_Phenytoin (Free) 14237
last_AT (Antithrombin funct) 14236
first_AT (Antithrombin funct) 14236
last_Tobramycin (Trough) 14235
first_Tobramycin (Trough) 14235
last_UF Heparin (Anti-Xa) 14234
first_UF Heparin (Anti-Xa) 14234
first_Tobramycin (Random) 14233
last_Tobramycin (Random) 14233
first_Gentamicin (Random) 14231
last_Gentamicin (Random) 14231
last_Gentamicin (Trough) 14231
first_Gentamicin (Trough) 14231
last_Thrombin 14224
first_Thrombin 14224
first_Total Granulocyte Count (TGC) 14215
last_Total Granulocyte Count (TGC) 14215
first_Creatinine (whole blood) 14212
last_Creatinine (whole blood) 14212
last_Phenobarbital 14202
first_Phenobarbital 14202
last_Absolute Neutrophil Count 14201
first_Absolu

In [99]:
cols_to_drop_labs = data_labs.isna().sum().sort_values(ascending=False).index[:119]
data_labs = data_labs.drop(columns=cols_to_drop_labs)

In [144]:
data_labs.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'first_ALT',
       'first_AST', 'first_Albumin', 'first_Alkaline Phosphate',
       'first_Anion gap', 'first_Arterial Base Excess',
       'first_Arterial CO2 Pressure', 'first_Arterial O2 pressure',
       'first_BUN', 'first_CK (CPK)', 'first_CK-MB',
       'first_Calcium non-ionized', 'first_Chloride (serum)',
       'first_Creatinine (serum)', 'first_Differential-Basos',
       'first_Differential-Eos', 'first_Differential-Lymphs',
       'first_Differential-Monos', 'first_Differential-Neuts',
       'first_Glucose (serum)', 'first_Glucose (whole blood)',
       'first_Glucose finger stick (range 70-100)', 'first_HCO3 (serum)',
       'first_Hematocrit (serum)', 'first_Hemoglobin', 'first_INR',
       'first_Ionized Calcium', 'first_LDH', 'first_Lactic Acid',
       'first_Magnesium', 'first_PH (Arterial)', 'first_PH (dipstick)',
       'first_PTT', 'first_Phosphorous', 'first_Platelet Count',
       'first_Potassium (serum

In [145]:
data_labs_columns = list(data_labs.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id']).columns)

### Vitals

In [101]:
data_vitals = data_vitals[data_vitals.stay_id.isin(stays)]
print(data_vitals.stay_id.unique().shape[0])

14256


In [102]:
for k,v in data_vitals.isna().sum().sort_values(ascending=False).iteritems():
    print(k,v)

max_GlucoseScore_ApacheIV 14256
last_Glucose_ApacheIV 14256
max_Glucose_ApacheIV 14256
first_Glucose_ApacheIV 14256
first_GlucoseScore_ApacheIV 14256
median_Glucose_ApacheIV 14256
median_GlucoseScore_ApacheIV 14256
min_GlucoseScore_ApacheIV 14256
min_Glucose_ApacheIV 14256
last_GlucoseScore_ApacheIV 14256
last_Rest HR - Aerobic Capacity 14253
first_Rest HR - Aerobic Capacity 14253
max_Rest HR - Aerobic Capacity 14253
median_Rest HR - Aerobic Capacity 14253
min_Rest HR - Aerobic Capacity 14253
median_rhr aar 14250
first_rhr aar 14250
max_rhr aar 14250
min_rhr aar 14250
last_rhr aar 14250
min_ohr standing 14230
min_obps standing 14230
median_ohr standing 14230
median_obps standing 14230
max_obps standing 14230
last_ohr standing 14230
last_obps standing 14230
max_ohr standing 14230
first_ohr standing 14230
first_obps standing 14230
median_ohr lying 14227
first_ohr sitting 14227
median_ohr sitting 14227
first_ohr lying 14227
last_ohr sitting 14227
min_ohr lying 14227
max_ohr sitting 14227


In [103]:
cols_to_drop_vitals = data_vitals.isna().sum().sort_values(ascending=False).index[:130]
data_vitals = data_vitals.drop(columns=cols_to_drop_vitals)

In [142]:
data_vitals.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_day_id', 'first_HR',
       'first_HR Alarm - High', 'first_HR Alarm - Low', 'first_NBPd',
       'first_NBPm', 'first_NBPs', 'first_Temperature F', 'last_HR',
       'last_HR Alarm - High', 'last_HR Alarm - Low', 'last_NBPd', 'last_NBPm',
       'last_NBPs', 'last_Temperature F', 'median_HR',
       'median_HR Alarm - High', 'median_HR Alarm - Low', 'median_NBPd',
       'median_NBPm', 'median_NBPs', 'median_Temperature F', 'min_HR',
       'min_HR Alarm - High', 'min_HR Alarm - Low', 'min_NBPd', 'min_NBPm',
       'min_NBPs', 'min_Temperature F', 'max_HR', 'max_HR Alarm - High',
       'max_HR Alarm - Low', 'max_NBPd', 'max_NBPm', 'max_NBPs',
       'max_Temperature F'],
      dtype='object')

In [143]:
data_vitals_columns = list(data_vitals.drop(columns=['subject_id', 'hadm_id', 'stay_id', 'icu_day_id']).columns)

### Medications

In [63]:
data_medications = data_medications[data_medications.stay_id.isin(stays)]
print(data_medications.stay_id.unique().shape[0])

14282


In [70]:
data_medications

,stay_id,M01,B01,C03,C02
2,31090461,0,1,0,0
3,32610785,0,0,0,0
5,35479615,0,1,0,0
6,33987268,0,1,0,0
7,38875437,0,0,0,0
...,...,...,...,...,...
36900,37403074,0,0,0,0
36901,31417783,0,1,1,1
36903,31144045,0,1,1,0
36906,30143796,0,0,0,0


In [79]:
data_medications_columns = list(data_medications.drop(columns='stay_id').columns)

### Demographics

In [118]:
data_labs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14246 entries, 9 to 75387
Data columns (total 93 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   subject_id                                 14246 non-null  uint64 
 1   hadm_id                                    14246 non-null  uint64 
 2   stay_id                                    14246 non-null  int64  
 3   icu_day_id                                 14246 non-null  int64  
 4   first_ALT                                  5946 non-null   float64
 5   first_AST                                  5943 non-null   float64
 6   first_Albumin                              3956 non-null   float64
 7   first_Alkaline Phosphate                   5829 non-null   float64
 8   first_Anion gap                            14065 non-null  float64
 9   first_Arterial Base Excess                 6694 non-null   float64
 10  first_Arterial CO2 Pre

In [137]:
data_demographics = dataset_demo[dataset_demo.stay_id.isin(stays)].drop_duplicates()

In [129]:
data_demographics.isna().sum()

subject_id          0
hadm_id             0
stay_id             0
race                0
gender              0
age                 0
BMI             10464
Height          10944
Weight          10160
diastolic_bp    10233
systolic_bp     10233
dtype: int64

In [149]:
dataset_demo_cols = list(dataset_demo.drop(columns=['subject_id', 'hadm_id', 'stay_id']).columns)

### Merge data

In [156]:
data = dataset_demo.merge(data_labs, on='stay_id', how='right').merge(data_vitals, on='stay_id', how='outer').merge(data_outputs, on=['stay_id', 'hadm_id'],how='outer').merge(data_medications, on='stay_id', how='left')

/home/svetlanamaslenkova/anaconda_envs/aki_env/lib/python3.7/site-packages/pandas/core/frame.py:9203: FutureWarning: Passing 'suffixes' which cause duplicate columns {'subject_id_x'} in the result is deprecated and will raise a MergeError in a future version.
  validate=validate,


In [157]:
train_data = data[data.stay_id.isin(train_stays)]
test_data = data[data.stay_id.isin(test_stays)]
val_data = data[data.stay_id.isin(val_stays)]

# Missing value inputation

In [158]:
def impute_missing(data, columns_to_norm):
    df = data.copy()
    for column in columns_to_norm:
        try:
            df[column] = df[column].fillna(np.round(df[column].mean(), 2))
        except:
            print(column)
            raise

    return df

In [148]:
data_labs_columns

Index(['first_ALT', 'first_AST', 'first_Albumin', 'first_Alkaline Phosphate',
       'first_Anion gap', 'first_Arterial Base Excess',
       'first_Arterial CO2 Pressure', 'first_Arterial O2 pressure',
       'first_BUN', 'first_CK (CPK)', 'first_CK-MB',
       'first_Calcium non-ionized', 'first_Chloride (serum)',
       'first_Creatinine (serum)', 'first_Differential-Basos',
       'first_Differential-Eos', 'first_Differential-Lymphs',
       'first_Differential-Monos', 'first_Differential-Neuts',
       'first_Glucose (serum)', 'first_Glucose (whole blood)',
       'first_Glucose finger stick (range 70-100)', 'first_HCO3 (serum)',
       'first_Hematocrit (serum)', 'first_Hemoglobin', 'first_INR',
       'first_Ionized Calcium', 'first_LDH', 'first_Lactic Acid',
       'first_Magnesium', 'first_PH (Arterial)', 'first_PH (dipstick)',
       'first_PTT', 'first_Phosphorous', 'first_Platelet Count',
       'first_Potassium (serum)', 'first_Potassium (whole blood)',
       'first_Prothr

In [159]:
train_data = impute_missing(train_data, [*data_vitals_columns, *data_labs_columns, *['output'], *dataset_demo_cols])
val_data = impute_missing(val_data, [*data_vitals_columns, *data_labs_columns, *['output'], *dataset_demo_cols])
test_data = impute_missing(test_data, [*data_vitals_columns, *data_labs_columns, *['output'], *dataset_demo_cols])

In [151]:
[*data_vitals_columns, *data_labs_columns, *['output'], *dataset_demo_cols]

['first_HR',
 'first_HR Alarm - High',
 'first_HR Alarm - Low',
 'first_NBPd',
 'first_NBPm',
 'first_NBPs',
 'first_Temperature F',
 'last_HR',
 'last_HR Alarm - High',
 'last_HR Alarm - Low',
 'last_NBPd',
 'last_NBPm',
 'last_NBPs',
 'last_Temperature F',
 'median_HR',
 'median_HR Alarm - High',
 'median_HR Alarm - Low',
 'median_NBPd',
 'median_NBPm',
 'median_NBPs',
 'median_Temperature F',
 'min_HR',
 'min_HR Alarm - High',
 'min_HR Alarm - Low',
 'min_NBPd',
 'min_NBPm',
 'min_NBPs',
 'min_Temperature F',
 'max_HR',
 'max_HR Alarm - High',
 'max_HR Alarm - Low',
 'max_NBPd',
 'max_NBPm',
 'max_NBPs',
 'max_Temperature F',
 'first_ALT',
 'first_AST',
 'first_Albumin',
 'first_Alkaline Phosphate',
 'first_Anion gap',
 'first_Arterial Base Excess',
 'first_Arterial CO2 Pressure',
 'first_Arterial O2 pressure',
 'first_BUN',
 'first_CK (CPK)',
 'first_CK-MB',
 'first_Calcium non-ionized',
 'first_Chloride (serum)',
 'first_Creatinine (serum)',
 'first_Differential-Basos',
 'first_Di